In [1]:
import os
import json
import joblib
import time

import pandas as pd
import random
from IPython.display import display, HTML
import torch
import numpy as np

In [2]:
# import torch
# import transformers
# from transformers import BartTokenizer, BartForConditionalGeneration

In [3]:
# #connect to hugging face
# !git clone https://github.com/huggingface/transformers \
# && cd transformers \
# install
#!pip install -q ./transformers

In [4]:
!pip install transformers==4.16.2

In [5]:
# # Installing transformers
# !pip install simpletransformers
# !pip install transformers
# # Importing necessary libraries from simple transformers
# import pandas as pd
# from simpletransformers.seq2seq import Seq2SeqModel,Seq2SeqArgs

In [6]:
# pip install ipywidgets

In [7]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print('Using: ', device)

In [8]:
# headlines_processed = []
# clean_news = []
# dataset = []
# target = []
# count = 0

In [9]:
path_models = './models/'
path_outputs = './outputs/'
path_processed_data= './processed_data/'

In [10]:
jsonFile = open(os.path.join(path_processed_data, 'cnn_train_v3.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_train = details_dict['text']
Y_train = details_dict['headlines']

jsonFile = open(os.path.join(path_processed_data, 'cnn_valid_v3.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_valid = details_dict['text']
Y_valid = details_dict['headlines']

jsonFile = open(os.path.join(path_processed_data, 'cnn_test_v3.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_test = details_dict['text']
Y_test = details_dict['headlines']


In [11]:
print(len(X_train), len(Y_train), len(X_valid), len(Y_valid), len(X_test), len(Y_test))

73280 73280 9160 9160 9160 9160


In [12]:
details = {
    'headlines' : Y_train,
    'text' : X_train
}

df_train = pd.DataFrame(details)

details = {
    'headlines' : Y_valid,
    'text' : X_valid
}

df_valid = pd.DataFrame(details)

details = {
    'headlines' : Y_test,
    'text' : X_test
}

df_test = pd.DataFrame(details)

In [13]:
# # Import
# import numpy as np # linear algebra
# import pandas as pd
# import logging
# import os

# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

# torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print('Using: ', torch_device)


In [14]:
import os, psutil  

def cpu_stats():
    pid = os.getpid()
    py = psutil.Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return 'memory GB:' + str(np.round(memory_use, 2))

def current_time_min():
    time_in_mins = time.time()/60.0
    return round(time_in_mins, 4)

In [15]:
cpu_stats()

'memory GB:0.63'

In [16]:
# import logging

# import pandas as pd
# from simpletransformers.t5 import T5Model

# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

In [17]:
df_train['prefix'] = "summarize"
df_test['prefix'] = "summarize"
df_valid['prefix'] = "summarize"

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 512,
    "train_batch_size": 16,
    "num_train_epochs": 10,
}



In [18]:
# Create T5 Model
#model = T5Model(model_type="t5", model_name="t5-base", args=model_args, use_cuda=True)

In [19]:
# Rename columns as per pretrained model required format
df_train=df_train.rename(columns={'headlines':'target_text','text':'source_text'})
df_valid=df_valid.rename(columns={'headlines':'target_text','text':'source_text'})
df_test=df_test.rename(columns={'headlines':'target_text','text':'source_text'})


In [20]:
df_train.shape, df_valid.shape, df_test.shape

((73280, 3), (9160, 3), (9160, 3))

In [21]:
# len(df_train)//10000

In [22]:
# start_time = current_time_min()
# model.train_model(df_train[:100])
# batch_time = current_time_min() - start_time
# print(batch_time)


In [23]:
!pip install simplet5

In [24]:
from simplet5 import SimpleT5

Global seed set to 42


In [25]:

model = SimpleT5()
#model.from_pretrained(model_type="t5", model_name="t5-base")
model.from_pretrained(model_type="t5", model_name="t5-large")

model.train(train_df=df_train,
            eval_df=df_valid, 
            source_max_token_len=128, 
            target_max_token_len=50, 
            batch_size=16, 
            max_epochs=10, 
            use_gpu=True
           )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 737 M 
-----------------------------------------------------
737 M     Trainable params
0         Non-trainable params
737 M     Total params
2,950.672 Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/ubuntu/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
/home/ubuntu/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 14.76 GiB total capacity; 13.45 GiB already allocated; 13.75 MiB free; 13.77 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
df_test['source_text'] = "summarize: " + df_test['source_text']

In [ ]:
df_test.head()

In [ ]:
# model_filename = "model_t5_3e.pkl"
# joblib.dump(model, os.path.join(path_models, model_filename))

In [ ]:
#model = joblib.load(os.path.join(path_models, 'model_t5_3e.pkl'))
model.load_model("t5", "outputs/simplet5-epoch-5-train-loss-2.2146-val-loss-2.8844", use_gpu=True)

In [ ]:
t5_prepared_Text = "summarize: " + df_test[0:1]['source_text'][0]
P1 = model.predict(t5_prepared_Text)

In [ ]:
P1

In [ ]:
#from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [ ]:
#model = T5ForConditionalGeneration.from_pretrained('t5-small')
#tokenizer = T5Tokenizer.from_pretrained(model_type="t5", model_name="t5-base", pretrained_model_name_or_path='./models/model_t5_3e.pkl')

t5_prepared_Text = "summarize: " + df_test[0:1]['source_text'][0]
#tokenized_text = tokenizer.encode(t5_prepared_Text, max_length=2048, return_tensors="pt", truncation=True).to(device) #default 512
        
summary_ids = model.predict(t5_prepared_Text,
                            num_beams=4,
                            max_length=60,
                            early_stopping=True,
                            use_gpu=True)

summary_t5 = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
# P1 = model.predict(df_test[0:1]['source_text'][0])
# #df_test[0:1]['source_text'][0]

In [ ]:
# Training the model and keeping eval dataset as test data
# 16000 for aws, 800 for local VM
#batch_size_train = 10000
batch_size_train = 100

batch_size_test = batch_size_train//4
number_of_batches = len(df_train)//batch_size_train
#count = 0
epocs = model_args['num_train_epochs']
#acc_time = 0

for i in range(number_of_batches):
    start_time = current_time_min()
    count += 1
    start_offset_train = i * batch_size_train
    start_offset_test = i * batch_size_test
    end_offset_train = start_offset_train + batch_size_train
    end_offset_test = start_offset_test + batch_size_test
    model.train_model(df_train[start_offset_train:end_offset_train], eval_data=df_valid)
    model_filename = "model_" + str(count) + "_" + str(start_offset_train) + "_" + str(end_offset_train) + "_" + str(epocs) + "e.pkl"
    joblib.dump(model, os.path.join(path_models, model_filename))
    results = model.eval_model(df_test)
    batch_time = current_time_min() - start_time
    acc_time = acc_time + batch_time/60
    randint = random.randrange(1, len(df_test), 3)
    results_json = {
        'iteration' : count,
        'start' : start_offset_train,
        'end' : end_offset_train,
        'eval_loss' : results["eval_loss"],
        'batch_time' : batch_time,
        'acc_time' : acc_time
    }
    jsonString = json.dumps(results_json)
    results_filename = "results_" + str(start_offset_train) + "_" + str(end_offset_train) + "_" + str(count)
    results_file = open(os.path.join(path_models, "results.txt"), "a")
    results_file.write(jsonString + '\n')
    results_file.close()
    print("Count: ", count, "Results: ", results)


In [ ]:
# Generating summaries on news test data
results = model.eval_model(df_test[100:200])
# print the loss
results

In [ ]:


# MODEL_PATH = os.path.join(path_models, 'bart.model')

# torch.save(model, MODEL_PATH)

In [ ]:
joblib.dump(model, os.path.join(path_models, 'model.pkl'))

In [ ]:
model = Seq2SeqModel(encoder_decoder_type="bart",
  encoder_decoder_name="facebook/bart-large",
  args=model_args,
  use_cuda=False,)

model.load(torch.load(MODEL_PATH))


In [ ]:
model = joblib.load(os.path.join(path_models, 'model.pkl'))

In [ ]:
# model.eval()

In [ ]:
# Original test data text summary for top 10 news
#value = randint(0, 10)
for i in test_df.target_text[422:423]:
  print(i)



In [ ]:
# Predicted summary
for i in test_df.input_text[1000:1050]:
  print(model.predict([i]))